# Final Project
## Remi LeBlanc and Max Shinnerl

https://github.com/fastai/fastbook/blob/master/12_nlp_dive.ipynb

Some info:

Original fastai model was pretrained with wiki and finetuned on these african new articles
Task to was a binary classification to determine in the tags 'Agribusiness' or 'Food and Agriculture' were in the list of tags for each article. One problem with this was only 3% of the articles had this tag, this made it a bit interesting. Accuracy is not a good metric, used f1/f beta score and precision and recall as our metric.
Fastai model performed about 80% recall 60% precision.

For our project we can either continue looking for agriculture news articles, becasue then we wont have to do anything with the fastai notebooks, or we can do a multilabel or multiclass somehow. 

The link above in from the fastai text book and he goes through out to make a lanugage model from scratch in pytorch - we might be able to follow that

### How will we compare the models?

We can just compare the metrics we chose - precision/recall and f1score.
We can talk about how it is harder to build from scratch, but have more flexibility. We should find something to do in the pytorch model that helps our model we couldnt do in fastai. Like perhaps with our unbalanced data

# Preprocessing
#### First get into a dataframe

In [1]:
# output_0628_fixedwithspace_updated.json has about 100,000 articles

In [2]:
# output_0728.json has about 500k with 241k in english

In [3]:
# small_data.json has 5000 articles

In [27]:
import json
import pandas as pd
import os
import re

import spacy
from tqdm.notebook import tqdm
from collections import Counter
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

In [5]:
#with open('output_0628_fixedwithspace_updated.json') as json_file:
#with open('output_0728.json') as json_file:
with open('small_data.json') as json_file:

    data = json.load(json_file)

In [6]:
len(data)

5000

In [7]:
all_articles = {key:val for key, val in data if type(val) == dict}


In [8]:
len(all_articles)

5000

In [9]:
all_articles = {k:v for k,v in all_articles.items() if v!='Article NA'}
all_articles = {k:v for k,v in all_articles.items() if v['language']=='English'}

In [10]:
len(all_articles)

5000

In [11]:
all_articles['202003010001']

{'title': 'Somalia: Sufi Muslim Leaders Surrender to Government',
 'article': ' The leaders of a Sufi Muslim group turned themselves into the custody of the Somali government Saturday after fighting left 22 people dead in central Somalia. Moallim Mohamud Sheikh, the spiritual leader, and Sheikh Mohamed Shakir, the chief of Ahlu-Sunna Wal-Jamaa (ASWJ), are in the custody of the Somali national army in the town of Dhusamareb after the group’s militias were overpowered in a battle with government forces. Dhusamareb is the administrative capital of Galmudug state. "Our security forces have ended the standoff and disarmed all ASWJ militias,” Osman Isse Nur, the spokesperson of the newly elected president, told VOA. Speaking in a video posted online, ASWJ chief Sheikh Shakir said his group ceded power to the Somali national army. "We agreed to end the fighting for the sake of the civilians. We agreed to hand over ASWJ militias to the commander general who will, in return, take responsibility

In [12]:
# Clean up non-matching column names
for k,v in all_articles.items():
    # Update tag to tags
    t = v.get('tag')
    if t:
        all_articles[k]['tags'] = t
        del all_articles[k]['tag']
        
    # Update full_story to full
    f = v.get('full text')
    if str(f)!='None':
        all_articles[k]['full_story'] = f
        del all_articles[k]['full text']

In [13]:
df = pd.DataFrame.from_dict(all_articles,orient='index')
df[df['full text'].notnull()]

,title,article,author,date,source,language,original_url,tags,full_story,full text


In [14]:
df = df.drop(['full text'],axis=1)
len(df)

5000

In [15]:
df['liststring'] = [','.join(map(str, l)) for l in df['tags']]

In [16]:
# Get the indices of the articles with agri tags
indices = []
for i, tags in enumerate(df['liststring']):
    if 'Agribusiness' in tags or 'Food and Agriculture' in tags:
        indices.append(i)
# make column of 0 and 1 in df for appearance of that tag
ies = [0]*df.shape[0]
for x in indices:
    ies[x]+=1
df['agri_label'] = ies

In [17]:
df['agri_label'] = ies

In [18]:
# 3% of articles have agri/foodag tag
sum(df['agri_label']==1), len(df), sum(df['agri_label']==1)/len(df)

(123, 5000, 0.0246)

In [19]:
df.head()

,title,article,author,date,source,language,original_url,tags,full_story,liststring,agri_label
202003010001,Somalia: Sufi Muslim Leaders Surrender to Gove...,The leaders of a Sufi Muslim group turned the...,By Abdulaziz Osman,1 March 2020,"Voice of America (Washington, DC)",English,https://www.voanews.com/africa/somalias-sufi-m...,"[Somalia, East Africa, Legal Affairs, Conflict...",False,"Somalia,East Africa,Legal Affairs,Conflict,Arm...",0
202003010002,Libya: UN-Mediated Political Talks on Libya En...,U.N.-mediated political talks aimed at resolv...,By Lisa Schlein,1 March 2020,"Voice of America (Washington, DC)",English,https://www.voanews.com/middle-east/un-mediate...,"[Libya, Conflict, External Relations, North Af...",False,"Libya,Conflict,External Relations,North Africa...",0
202003010003,Nigeria: Obasanjo At 83 - a Leader and His Cou...,"""I don't want to be remembered. I am still he...",N/A,1 March 2020,This Day (Lagos),English,https://www.thisdaylive.com/index.php/2020/03/...,"[Nigeria, West Africa, Governance]",False,"Nigeria,West Africa,Governance",0
202003010004,"Sudan: Service of 14 Ambassadors, 35 Diplomats...",Khartoum — The Higher Committee for the disma...,N/A,29 February 2020,Sudan News Agency (Khartoum),English,https://suna-sd.net/en/single?id=561889,"[Sudan, East Africa, Governance]",False,"Sudan,East Africa,Governance",0
202003010005,Sudan: German President Ends Sudan Visit With ...,Khartoum — German President Frank-Walter Stei...,N/A,29 February 2020,Radio Dabanga (Amsterdam),English,https://www.dabangasudan.org/en/all-news/artic...,"[Sudan, External Relations, East Africa, Gover...",False,"Sudan,External Relations,East Africa,Governanc...",0


 We will build a language model from the 'article' column to predict the 'agri_label' column or the 'tags' column

#### Once in dataframe, let's do tokenization and build word vocab

In [20]:
# load information about words
#!python -m spacy download en_core_web_sm
nlp = spacy.load('en_core_web_sm')

In [21]:
# "article" column has article text
# Let's make new column with title and article text appended + tokenized and cleaned.
# also get max article length while we're here, for padding later

combined_clean = []
max_length = 0

for i in tqdm(range(len(df))):
    row = df.iloc[i].copy()
    combined = row['title'] + ' ' + row['article']
    combined = re.sub("[^A-Za-z']+", ' ', combined).lower()
    combined = nlp(combined)
    combined = [token.lemma_ for token in combined if ((not token.is_stop) or (' ' in token.text))]
    if len(combined) > 1:
        if len(combined) > max_length:
            max_length = len(combined)
        cleaned = ' '.join(combined)
    combined_clean.append(cleaned)

df['cleaned'] = combined_clean
df.to_csv('cleaned.csv')
print('max_length:', max_length)


max_length: 3575


#### re-read clean data and let's begin

In [22]:
df_clean = pd.read_csv('cleaned.csv')
df_clean.columns
max_length=3575 # calculated above, hardcoded here to avoid running long code

#### Get vocab

In [23]:
articles = [article.split(' ') for article in list(df_clean['cleaned'])]
word_freq = dict(Counter([token for text in articles for token in text]).most_common())
len(word_freq)

48338

In [24]:
min_freq = 5  # word must appear at least min_freq times to get it's own token

word2idx = {}
i = 1  # leave 0 for unknowns/infrequents
for word in word_freq:
    if word_freq[word] > min_freq:
        word2idx[word] = i
        i += 1
    else:
        word2idx[word] = 0

vocab_length = max(word2idx.values()) + 1
vocab_length  # essentially number of all unique words excluding infrequents

14424

In [25]:
class ArticleDataset(Dataset):
    def __init__(self, df, word_dict, max_length):
        self.df = df
        self.word_dict = word_dict
        self.max_length = max_length
        
    def __len__(self):
        return(len(self.df))
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx].copy()
        
        article = row['cleaned'].split(' ')
        x = torch.zeros(self.max_length)
        for i in range(len(article)):
            # front pad
            x[self.max_length - len(article) + i] = self.word_dict[article[i]]
            
        y = torch.tensor(row['agri_label'])
        
        return x.long(), y.float()

ds = ArticleDataset(df_clean, word2idx, max_length)
next(iter(ds))

(tensor([   0,    0,    0,  ..., 6502,  807, 1628]), tensor(0.))

In [29]:
# get data loaders
batch_size = 10

train_df, valid_df = train_test_split(df_clean, test_size=0.2)

train_ds = ArticleDataset(train_df, word2idx, max_length)
valid_ds = ArticleDataset(valid_df, word2idx, max_length)

train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=batch_size)

# Models

In [73]:
# LSTM Model 1
class LSTMModel(nn.Module):
    def __init__(self,
                 batch_size,
                 vocab_size=vocab_length,
                 hidden_size=50):
        
        super(LSTMModel, self).__init__()
        
        self.emb = nn.Embedding(vocab_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.h2o = nn.Linear(hidden_size, 1)  # binary classification
        
    
    def forward(self, x):
        emb = self.emb(x)
        out, hidden = self.lstm(emb)
        
        # many to one, so final hidden state through linear layer
        final = torch.squeeze(hidden[-1].permute((1,2,0)))
        final = self.h2o(final)
        
        
        return(torch.squeeze(final))

In [93]:
# let's train
epochs = 3
model = LSTMModel(batch_size)
lossFun = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

for epoch in tqdm(range(epochs)):
    total_loss = 0.0
    model.train()
    for x, y in tqdm(train_dl):
        yhat = model(x)
        
        loss = lossFun(yhat, y)
        
        total_loss += loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    avg_loss = total_loss / len(train_dl)
    print('epoch:', epoch, 'avg_loss:', avg_loss)
        
    

KeyboardInterrupt: 

#### Precision and recall:
Precision = True Positives / (True Positives + False Positives)

Recall = True Postivies / (True Positives + False Negatives)

Reason: Significantly fewer agriculture articles compared to others, don't really care about True Negatives (i.e. y and yhat both 0)

We want to emphasize guessing the agriculture ones correctly


In [91]:
def precision_recall(ytrues, ypreds):
    assert(len(ytrues) == len(ypreds))
    
    # don't need all of these, tracking just in case
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    N = len(ytrues)
    
    for ytrue, ypred in zip(ytrues, ypreds):
        
        if   ytrue == 1 and ypred == 1:
            TP += 1
        elif ytrue == 0 and ypred == 1:
            FP += 1
        elif ytrue == 1 and ypred == 0:
            FN += 1
        elif ytrue == 0 and ypred == 0:
            TN += 1
        else:
            print('bad input, not ones and zeroes')
            break
    
    if TP == 0:
        return (0,0)
    
    precision = TP / (TP + FP)
    recall    = TP / (TP + FN)
    
    return (precision,recall)

In [92]:
x, y = next(iter(valid_dl))
model.eval()
yhat = (torch.sigmoid(model(x)) > 0.5).float()
precision_recall(y, yhat)

(0, 0)